In [2]:

import stanza

import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
import pickle 
import re
import os
from tqdm import tqdm
from pathlib import Path
from os.path import expanduser
import json
import functools
import operator


In [3]:
import sys
home = expanduser("~")

# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, home+'/Documents/detect_hedges/')

from hedge_detection import isHedgedSentence

2022-08-23 11:46:15 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-08-23 11:46:15 INFO: Use device: gpu
2022-08-23 11:46:15 INFO: Loading: tokenize
2022-08-23 11:46:18 INFO: Loading: pos
2022-08-23 11:46:18 INFO: Loading: lemma
2022-08-23 11:46:18 INFO: Loading: depparse
2022-08-23 11:46:18 INFO: Loading: sentiment
2022-08-23 11:46:19 INFO: Loading: constituency
2022-08-23 11:46:19 INFO: Loading: ner
2022-08-23 11:46:20 INFO: Done loading processors!


In [4]:


DATA_DIR = home+'/Documents/spinos-project/data/' 
DATA_FN = 'spinos_user_history.pkl' 
with open(os.path.join(DATA_DIR, DATA_FN), "rb") as f:
     df_hist = pickle.load(f)


DATA_DIR = home+'/Documents/spinos-project/data/' 
DATA_FN = 'spinos_annotated.pkl'
with open(os.path.join(DATA_DIR, DATA_FN), "rb") as f:
     df = pickle.load(f)
        
        
DATA_DIR = home+'/Documents/spinos-project/data/' 
DATA_FN = 'spinos_context.pkl'
with open(os.path.join(DATA_DIR, DATA_FN), "rb") as f:
     df_cont = pickle.load(f)

In [5]:
from hedge_detection_c import Load_Lexicons, Detect_Hedges
ll = Load_Lexicons()
lexicons = ll.load()

dh = Detect_Hedges(lexicons)



2022-08-23 11:46:29 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-08-23 11:46:29 INFO: Use device: gpu
2022-08-23 11:46:29 INFO: Loading: tokenize
2022-08-23 11:46:29 INFO: Loading: pos
2022-08-23 11:46:30 INFO: Loading: lemma
2022-08-23 11:46:30 INFO: Loading: depparse
2022-08-23 11:46:30 INFO: Loading: sentiment
2022-08-23 11:46:30 INFO: Loading: constituency
2022-08-23 11:46:31 INFO: Loading: ner
2022-08-23 11:46:31 INFO: Done loading processors!


In [6]:
nlp = stanza.Pipeline(lang='en', processors='tokenize')

def hedge_percentage(paragraph):
    doc = nlp(paragraph)
    list_hedges = []
    for i, sentence in enumerate(doc.sentences):
        list_hedges.append(isHedgedSentence(sentence.text))
    
    return sum(list_hedges)/len(list_hedges)
    

2022-08-23 11:46:33 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-08-23 11:46:33 INFO: Use device: gpu
2022-08-23 11:46:33 INFO: Loading: tokenize
2022-08-23 11:46:33 INFO: Done loading processors!


In [7]:

import time
start = time.process_time()

list_hedges = []
for i, row in tqdm(df.head(100).iterrows()):
    x = row['all_text']
    try:
        list_hedges.append(hedge_percentage(x))
    except:
        print(x)
        list_hedges.append(np.nan)

print(time.process_time() - start)

# df['hedge_percentage'] = list_hedges
# df['hedge_percentage'] = df['hedge_percentage'].fillna(0)


100it [00:50,  1.98it/s]

50.444102654999995


In [ ]:
import time
start = time.process_time()

list_hedges = []
for i, row in tqdm(df.head(100).iterrows()):
    x = row['all_text']
    try:
        list_hedges.append(dh.hedge_percentage(x))
    except:
        print(x)
        list_hedges.append(np.nan)

print(time.process_time() - start)

7it [00:03,  1.25it/s]

In [10]:
# DATA_DIR = home+'/Documents/spinos-project/data/' 
# DATA_FN = 'spinos_annotated.pkl'

# df.to_pickle(DATA_DIR+DATA_FN)